In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import preprocessing,svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))
raw_df=pd.read_csv("../input/train.csv")





# Prepare column names 

In [ ]:
# select functions
def is_bin(s):
    return ("bin" in s)
def is_cat(s):
    return ("cat" in s)
def is_val(s):
    return not (is_bin(s) or is_cat(s))

# filter column names
cols = list(raw_df.columns)[2:]
cat_cols=list(filter(is_cat, cols))
bin_cols=list(filter(is_bin, cols))
val_cols=list(filter(is_val, cols))


# Prepare columns

In [ ]:
# preparing binary columns
bin_df=raw_df[bin_cols]

# preparing value columns
raw_val_df=raw_df[val_cols]
val_scaler = preprocessing.StandardScaler()
val_scaler.fit(raw_val_df)
val_df=pd.DataFrame(val_scaler.transform(raw_val_df))

# preparing category columns
raw_cat_df=raw_df[cat_cols]
cat_imputer = preprocessing.Imputer(missing_values=-1, strategy='most_frequent', axis=0)
cat_imputer.fit(raw_cat_df)
raw_cat_ne_df = cat_imputer.transform(raw_cat_df)
cat_one_hot_encoder = preprocessing.OneHotEncoder(sparse=False)
cat_one_hot_encoder.fit(raw_cat_ne_df)
cat_df = pd.DataFrame(cat_one_hot_encoder.transform(raw_cat_ne_df))

ans_df = raw_df[['target']]
# join dfs
df = ans_df.join(bin_df.join(cat_df.join(val_df, lsuffix="_cat",rsuffix="_val"),lsuffix="_bin",rsuffix="_nbi"))




In [ ]:
train_df=df
feat_cols = list(train_df.columns)[1:]
train_feat=train_df[feat_cols]
train_ans=train_df[["target"]]
# Any results you write to the current directory are saved as output.

In [ ]:
clf= RFE(RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1,class_weight='balanced'),20)
clf.fit(train_feat,train_ans.values.ravel())

In [ ]:
cv_df=df.sample(frac=0.02)
cv_feat=cv_df[feat_cols]
cv_ans=cv_df[["target"]]
clf.score(cv_feat,cv_ans.values.ravel())

In [ ]:
raw_df=pd.read_csv("../input/test.csv")
# preparing binary columns
bin_df=raw_df[bin_cols]

# preparing value columns
raw_val_df=raw_df[val_cols]
val_df=pd.DataFrame(val_scaler.transform(raw_val_df))

# preparing category columns
raw_cat_df=raw_df[cat_cols]
raw_cat_ne_df = cat_imputer.transform(raw_cat_df)
cat_df = pd.DataFrame(cat_one_hot_encoder.transform(raw_cat_ne_df))

#ans_df = raw_df[['target']]
# join dfs
df = bin_df.join(cat_df.join(val_df, lsuffix="_cat",rsuffix="_val"),lsuffix="_bin",rsuffix="_nbi")


In [ ]:
pred = clf.predict(df)
print(df.shape)
print(df.head())

In [ ]:
pred.shape

In [ ]:
(raw_df[['id']].join(pd.DataFrame(pred, columns=['target']))).to_csv("./outie.csv",index=False)